In [1]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [5]:
data = wine[['alcohol', 'sugar', 'pH']]
target = wine[['class']]

In [8]:
from sklearn.model_selection import train_test_split #섞는 기능이 내제되어있는 메써드?

train_input, test_input, train_target, test_target = \
train_test_split(data, target, random_state=42)

In [12]:
sub_input, val_input, sub_target, val_target = \
train_test_split(train_input, train_target, random_state=42) #검증셋트

In [14]:
print(data.shape, train_input.shape, test_input.shape)
print(sub_input.shape, val_input.shape)

(6497, 3) (4872, 3) (1625, 3)
(3654, 3) (1218, 3)


In [15]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9978106185002736
0.8571428571428571


In [16]:
#3폴드 교차 검증

In [18]:
from sklearn.model_selection import cross_validate
#train_input (전체 데이터의 80%)
#test셋은 실전용 데이터로 따로 빼두는 개념 (pure) 검증?
scores = cross_validate(dt, train_input, train_target)
scores

{'fit_time': array([0.0069983 , 0.00599957, 0.00704837, 0.00595427, 0.00803018]),
 'score_time': array([0.00299716, 0.00300455, 0.00199318, 0.0040288 , 0.0019424 ]),
 'test_score': array([0.85128205, 0.84820513, 0.8788501 , 0.85112936, 0.84394251])}

In [21]:
scores['test_score']

array([0.85128205, 0.84820513, 0.8788501 , 0.85112936, 0.84394251])

In [23]:
import numpy as np
np.mean(scores['test_score']) #주의!: 검증폴드점수

np.float64(0.8546818301479492)

In [27]:
from sklearn.model_selection import StratifiedKFold #섞어주는 놈

spliter = StratifiedKFold(n_splits=10, shuffle=True)
scores = cross_validate(dt, train_input, train_target, cv=spliter)
scores

{'fit_time': array([0.00692487, 0.00599837, 0.00700068, 0.0059998 , 0.00599504,
        0.00604129, 0.00702095, 0.0059917 , 0.00600433, 0.00599957]),
 'score_time': array([0.00200057, 0.00199986, 0.00303817, 0.00200319, 0.00200629,
        0.00200081, 0.00198102, 0.00199771, 0.00200486, 0.00100088]),
 'test_score': array([0.89139344, 0.86065574, 0.87679671, 0.84804928, 0.83983573,
        0.87679671, 0.87063655, 0.87474333, 0.862423  , 0.84804928])}

In [28]:
# 검증 배우는 이유? > 비정형 데이터에 대해 정답률을 높이기 위해 사용

In [29]:
from sklearn.model_selection import GridSearchCV

In [30]:
params = {
    'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]
}

In [33]:
dt = DecisionTreeClassifier(random_state=42)
gs = GridSearchCV(dt, params)

In [34]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [35]:
gs.best_estimator_

DecisionTreeClassifier(min_impurity_decrease=0.0003, random_state=42)

In [36]:
gs.best_params_

{'min_impurity_decrease': 0.0003}

In [39]:
gs.cv_results_

{'mean_fit_time': array([0.00517964, 0.00440717, 0.00399652, 0.00340276, 0.00359535]),
 'std_fit_time': array([4.12603614e-04, 4.86312493e-04, 5.02062204e-06, 4.88675632e-04,
        4.90962963e-04]),
 'mean_score_time': array([0.00233259, 0.00179892, 0.00140238, 0.0017909 , 0.00140347]),
 'std_score_time': array([0.00105435, 0.00039938, 0.00048785, 0.0004165 , 0.00048706]),
 'param_min_impurity_decrease': masked_array(data=[0.0001, 0.0002, 0.0003, 0.0004, 0.0005],
              mask=[False, False, False, False, False],
        fill_value=1e+20),
 'params': [{'min_impurity_decrease': 0.0001},
  {'min_impurity_decrease': 0.0002},
  {'min_impurity_decrease': 0.0003},
  {'min_impurity_decrease': 0.0004},
  {'min_impurity_decrease': 0.0005}],
 'split0_test_score': array([0.87384615, 0.87076923, 0.87282051, 0.86461538, 0.86051282]),
 'split1_test_score': array([0.86666667, 0.86871795, 0.87794872, 0.88512821, 0.87794872]),
 'split2_test_score': array([0.88603696, 0.88295688, 0.8798768 , 0.87

In [45]:
import numpy as np

params = {
    'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),
    'max_depth' :range(5, 20, 1),
    'min_samples_split' :range(2, 100, 10),
}

In [46]:
print(np.arange(0.0001, 0.001, 0.0001))
range(5, 20, 1)
range(2, 100, 10)

[0.0001 0.0002 0.0003 0.0004 0.0005 0.0006 0.0007 0.0008 0.0009]


range(2, 100, 10)

In [48]:
dt = DecisionTreeClassifier(random_state=42)

gs = GridSearchCV(dt, params, n_jobs=-1)

In [49]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [50]:
gs.best_params_

{'max_depth': 15,
 'min_impurity_decrease': np.float64(0.0001),
 'min_samples_split': 22}

In [52]:
gs.cv_results_['mean_test_score']

array([0.85837161, 0.85837161, 0.85837161, ..., 0.86309693, 0.86309693,
       0.86309693])

In [53]:
# scipy의 stats 서브 패키지에 있는 uniform과 randint는 모두 균등분포에서 샘플링한다고 함.
from scipy.stats import uniform, randint

In [72]:
# 0~9까지 정수 1000개 랜덤생성
rgen = randint(0, 10)
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 98, 110,  91,  80, 112, 105,  99,  98,  99, 108]))

In [74]:
#uniform은 실수 randint는 정수
params = {
    'min_impurity_decrease' : uniform(0.0001,0.001),
    'max_depth' : randint(20, 50),
    'min_samples_split' : randint(2, 25),
    'min_samples_leaf' : randint(1, 25),
}


In [75]:
from sklearn.model_selection import RandomizedSearchCV

In [76]:
gs = RandomizedSearchCV(dt, params, n_jobs=-1, n_iter=100)
gs.fit(train_input, train_target)

C:\Users\2-13\Desktop\DMF\ML\venv\Lib\site-packages\numpy\ma\core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x00000226E6A134D0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x00000226E5309730>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x00000226E68EBEC0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x00000226E66B0920>})

In [78]:
gs.best_estimator_

DecisionTreeClassifier(max_depth=20,
                       min_impurity_decrease=np.float64(0.00023320947492788094),
                       min_samples_leaf=5, min_samples_split=21,
                       random_state=42)

In [79]:
gs.best_params_

{'max_depth': 20,
 'min_impurity_decrease': np.float64(0.00023320947492788094),
 'min_samples_leaf': 5,
 'min_samples_split': 21}